In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r sample_data/

# setup spark

In [ ]:
#@title spark install { form-width: "20%" }
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pyspark


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [1]:
#@title spark setup { form-width: "20%" }

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "10g").getOrCreate()
sc = spark.sparkContext

# collect data

In [ ]:
# download the dataset
!wget https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline.tar.gz

--2022-01-25 13:26:07--  https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline.tar.gz
Resolving dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)... 104.112.1.31, 2600:1408:20:29c::d2d, 2600:1408:20:293::d2d
Connecting to dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)|104.112.1.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7163404119 (6.7G) [application/x-gzip]
Saving to: ‘airline.tar.gz’

airline.tar.gz      100%[===================>]   6.67G  6.12MB/s    in 19m 38s 

2022-01-25 13:45:45 (5.80 MB/s) - ‘airline.tar.gz’ saved [7163404119/7163404119]



In [ ]:
!tar -xvf airline.tar.gz

In [ ]:
cd drive/MyDrive/BigData-finalproject/complete_data/subset/

/content/drive/MyDrive/BigData-finalproject/complete_data/subset


In [ ]:
# spit huge 58 gb csv to smaller csvs of 22 million records each 
!split -d -l 22000000 airline.csv part.

split: part.05: No space left on device


In [ ]:
# add the headers
%%bash 
for file in part.*
do
    head -n 1 airline.csv > tmp_file
    cat "$file" >> tmp_file
    mv -f tmp_file "$file"
done

### note: i later moved these to subset/raw segments

# take subset from data, remove unnecessary columns

i ran 5 notebooks in parallel, one for each segment file. this is because the ssd disk space of colab kept getting overwhelmed with the whole 58 GB of data

In [ ]:
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/subset/part.00')


In [ ]:
columns= ['Year',
          'Month',
          'DayofMonth',
          'DayOfWeek',
          'Reporting_Airline',
          'Tail_Number',
          'Flight_Number_Reporting_Airline',
          'OriginAirportID',
          'Origin',
          'OriginCityName',
          'OriginState',
          'OriginStateName',
          'DestAirportID',
          'Dest',
          'DestCityName',
          'DestState',
          'DestStateName',
          'DepTime',
          'DepDelay',
          'ArrTime',
          'ArrDelay',
          'Cancelled',
          'Diverted',
          'ActualElapsedTime',
          'Flights',
          'Distance',
          'CarrierDelay',
          'WeatherDelay',
          'NASDelay',
          'SecurityDelay',
          'LateAircraftDelay'
]

df2 = df.select(columns)

# save selected subset

In [ ]:
# 4 more copies of this notebook will be processing parts 1, 2, 3, and 4
df2.write.csv('part00.csv')

saving a csv with spark's write csv utility saves it in 64 segments. in the following cells, I merge the segments into one file for part00

In [ ]:
cd part00.csv

/content/part00.csv


In [ ]:
%%bash
awk '(NR == 1) || (FNR > 1)'  part00.csv/*.csv > part-00.csv
cp part00.csv/part-00.csv  drive/MyDrive/BigData-finalproject/complete_data

In [ ]:
!cp part00.csv/part-00.csv head drive/MyDrive/BigData-finalproject/complete_data

### now i'll merge the five parts 00:04 into one complete csv. dropping unneccary columns reduced the data size from 58 to 16 GB

In [ ]:
cd drive/MyDrive/BigData-finalproject/complete_data


/content/drive/MyDrive/BigData-finalproject/complete_data


In [ ]:
# the new header
",".join(columns)

'Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay'

In [ ]:
#merge
%%bash
awk '(NR == 1) || (FNR > 1)' *.csv > merged.csv



In [ ]:
#add the new header
!sed  -i '1i Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay' merged.csv


In [ ]:
!head -n 2 merged.csv

Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1987,10,30,5,PS,"",1442,13796,OAK,"Oakland, CA",CA,California,10800,BUR,"Burbank, CA",CA,California,936,-1.0,1046,6.0,0.0,0.0,70.0,1.0,325.0,"","","","",""


# data cleaning


In [ ]:
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/merged.csv').persist()


In [ ]:
df.show()

+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+------------+---------+-------------+-------+--------+-------+--------+---------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|Reporting_Airline|Tail_Number|Flight_Number_Reporting_Airline|OriginAirportID|Origin|OriginCityName|OriginState|OriginStateName|DestAirportID|Dest|DestCityName|DestState|DestStateName|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|Diverted|ActualElapsedTime|Flights|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+------------+---------+-------------+-------+--------+-------+--------+---

In [ ]:
df.summary().show()

+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+-------------------------------+------------------+---------+--------------+-----------+---------------+------------------+---------+------------+---------+-------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+---------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|              Year|            Month|        DayofMonth|         DayOfWeek|Reporting_Airline|       Tail_Number|Flight_Number_Reporting_Airline|   OriginAirportID|   Origin|OriginCityName|OriginState|OriginStateName|     DestAirportID|     Dest|DestCityName|DestState|DestStateName|           DepTime|          DepDelay|           ArrTime|         ArrDelay|           Cancelled|            Diverted| ActualElapsedTime|  Flights|         Di

In [ ]:
from pyspark.sql.functions import *
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+------------+---------+-------------+-------+--------+-------+--------+---------+--------+-----------------+-------+--------+------------+------------+---------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|Reporting_Airline|Tail_Number|Flight_Number_Reporting_Airline|OriginAirportID|Origin|OriginCityName|OriginState|OriginStateName|DestAirportID|Dest|DestCityName|DestState|DestStateName|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|Diverted|ActualElapsedTime|Flights|Distance|CarrierDelay|WeatherDelay| NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-----------------+-----------+-------------------------------+---------------+------+--------------+-----------+---------------+-------------+----+------------+---------+-------------+-------+--------+-------+--------+-

In [ ]:
#columns with 104 million nulls:
too_many_nulls= ('CarrierDelay','WeatherDelay', 'NASDelay','SecurityDelay','LateAircraftDelay')
df = df.drop(*too_many_nulls)

In [ ]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCityName: string (nullable = true)
 |-- DestState: string (nullable = true)
 |-- DestStateName: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- Diverted: double (nullable = true)
 |--

In [ ]:
# now we can drop nulls safely and expect to lose no more that 37  million records out of 110 million
# drop nulls, nans, and empty strings
def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))

df2 = df.select([to_null(c).alias(c) for c in df.columns]).na.drop()
df2.count()

70992524

In [ ]:
pwd

'/content'

In [ ]:
# save the new clean df
df2.write.csv('clean.csv')

In [ ]:
# Merge the output
%%bash
awk '(NR == 1) || (FNR > 1)'  clean.csv/*.csv >clean_data.csv


In [ ]:
# check out the header
!head -n 2 clean_data.csv

1995,1,26,4,AA,N616AA,521,13303,MIA,"Miami, FL",FL,Florida,15024,STT,"Charlotte Amalie, VI",VI,U.S. Virgin Islands,1718,13.0,2050,8.0,0.0,0.0,152.0,1.0,1107.0
1995,1,27,5,AA,N5DFAA,521,13303,MIA,"Miami, FL",FL,Florida,15024,STT,"Charlotte Amalie, VI",VI,U.S. Virgin Islands,1707,2.0,2044,2.0,0.0,0.0,157.0,1.0,1107.0


In [ ]:
','.join(df2.columns)

'Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance'

In [ ]:
#add the header
!sed  -i '1i Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance' clean_data.csv


In [ ]:
!head -n 2 clean_data.csv

Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance
1995,1,26,4,AA,N616AA,521,13303,MIA,"Miami, FL",FL,Florida,15024,STT,"Charlotte Amalie, VI",VI,U.S. Virgin Islands,1718,13.0,2050,8.0,0.0,0.0,152.0,1.0,1107.0


In [ ]:
# move to drive
!cp clean_data.csv drive/MyDrive/BigData-finalproject/complete_data

# exploring the dataframe

In [ ]:
df2 = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/BigData-finalproject/complete_data/clean_data.csv').persist()


In [ ]:
df2.groupBy('Year').count().show()

+----+-------+
|Year|  count|
+----+-------+
|2003|6375599|
|2007| 907251|
|2006|7003613|
|1997|5301975|
|2004|6987150|
|1996|5209305|
|1998|5227026|
|1995|5225015|
|2001|5723646|
|2005|6992750|
|2000|5481278|
|1999|5359994|
|2002|5197835|
+----+-------+



In [ ]:
df2.groupBy('OriginCityName').count().show()

+--------------------+------+
|      OriginCityName| count|
+--------------------+------+
|     Gainesville, FL| 13394|
|         Ontario, CA|430972|
|        Richmond, VA|195969|
|          Tucson, AZ|229873|
|    Myrtle Beach, SC| 47797|
|         Medford, OR| 30648|
|    Palm Springs, CA| 84824|
|         Redding, CA|  8228|
|         Durango, CO|  8071|
|        Gustavus, AK|  1079|
|          Mobile, AL| 51719|
|  Corpus Christi, TX| 65563|
|         Dubuque, IA|  7331|
|        Columbus, GA|  8192|
|      Huntsville, AL| 82373|
|       Pensacola, FL| 81154|
|      Fort Myers, FL|213698|
|Pasco/Kennewick/R...| 23499|
|     Springfield, IL|  2100|
|           Aspen, CO|  4096|
+--------------------+------+
only showing top 20 rows



# getting the 2020 data from the american bureau of transportation website

In [11]:
import zipfile
import os
import glob 
filenames = glob.glob('drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/*')

In [12]:
filenames

['drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-jan.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-feb.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-march.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/202-april.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-may.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-june.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-july.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-aug.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-oct.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-nov.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2020-dec.zip',
 'drive/MyDrive/BigData-finalproject/complete_data/subset/2019&2020/2019-jan.zip',

In [13]:
for FILE in filenames:
    z = zipfile.ZipFile(FILE)
    name = FILE.split('/')[-1].split('.')[0]+'.csv'
    for f in z.filelist:
        f.filename = name
    z.extract(f)

In [2]:
# concatenate all csv files into one
import pandas as pd 
import glob
csv_files = glob.glob('*.csv')
csv_files

['2019-aug.csv',
 '2019-jan.csv',
 '2020-june.csv',
 '202-april.csv',
 '2020-jan.csv',
 '2019-nov.csv',
 '2020-aug.csv',
 '2019-july.csv',
 '2019-feb.csv',
 '2020-oct.csv',
 '2020-march.csv',
 '2020-may.csv',
 'orig.csv',
 '2019-may.csv',
 '2019-mar.csv',
 '2019-apr.csv',
 '2019-sept.csv',
 'dst.csv',
 '2020-july.csv',
 '2019-june.csv',
 '2020-dec.csv',
 '2019-oct.csv',
 '2020-nov.csv',
 '2020-sept.csv',
 '2020-feb.csv',
 '2019-dec.csv']

In [ ]:
len(csv_files)

24

In [3]:
csv_files.remove('dst.csv')
csv_files.remove('orig.csv')

In [4]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in csv_files ])
#export to csv
#combined_csv.to_csv( "'drive/MyDrive/BigData-finalproject/complete_data/2019_2020_data.csv", index=False, encoding='utf-8-sig')

In [5]:
combined_csv.drop('Unnamed: 24', axis=1, inplace=True)

In [ ]:
combined_csv.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE
0,2019,2,1,5,AA,N110AN,1,12478,JFK,"New York, NY",NY,12892,LAX,"Los Angeles, CA",CA,759.0,-1.0,1058.0,-35.0,0.0,0.0,359.0,1.0,2475.0
1,2019,2,2,6,AA,N116AN,1,12478,JFK,"New York, NY",NY,12892,LAX,"Los Angeles, CA",CA,849.0,49.0,1156.0,23.0,0.0,0.0,367.0,1.0,2475.0
2,2019,2,3,7,AA,N105NN,1,12478,JFK,"New York, NY",NY,12892,LAX,"Los Angeles, CA",CA,756.0,-4.0,1108.0,-25.0,0.0,0.0,372.0,1.0,2475.0
3,2019,2,4,1,AA,N108NN,1,12478,JFK,"New York, NY",NY,12892,LAX,"Los Angeles, CA",CA,756.0,-4.0,1126.0,-7.0,0.0,0.0,390.0,1.0,2475.0
4,2019,2,5,2,AA,N110AN,1,12478,JFK,"New York, NY",NY,12892,LAX,"Los Angeles, CA",CA,755.0,-5.0,1112.0,-21.0,0.0,0.0,377.0,1.0,2475.0


In [ ]:
combined_csv.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST_AIRPORT_ID', 'DEST',
       'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEP_TIME', 'DEP_DELAY', 'ARR_TIME',
       'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'ACTUAL_ELAPSED_TIME', 'FLIGHTS',
       'DISTANCE'],
      dtype='object')

In [6]:
combined_csv.rename(columns={'YEAR':'Year', 'MONTH': 'Month', 'DAY_OF_MONTH' :'DayofMonth',
        'DAY_OF_WEEK':'DayOfWeek', 'OP_UNIQUE_CARRIER':'Reporting_Airline',
        'TAIL_NUM':'Tail_Number', 'OP_CARRIER_FL_NUM':'Flight_Number_Reporting_Airline',
        'ORIGIN_AIRPORT_ID':'OriginAirportID', 'ORIGIN': 'Origin',
        'ORIGIN_CITY_NAME': 'OriginCityName', 'ORIGIN_STATE_ABR': 'OriginState',
        'DEST_AIRPORT_ID': 'DestAirportID', 'DEST':'Dest',
        'DEST_CITY_NAME':'DestCityName', 'DEST_STATE_ABR': 'DestState',
        'DEP_TIME':'DepTime', 'DEP_DELAY': 'DepDelay', 'ARR_TIME':'ArrTime',
        'ARR_DELAY':'ArrDelay', 'CANCELLED': 'Cancelled', 'DIVERTED':'Diverted', 
        'ACTUAL_ELAPSED_TIME': 'ActualElapsedTime', 'FLIGHTS':'Flights',
        'DISTANCE':'Distance'}, inplace=True)

In [7]:
orig = pd.read_csv('orig.csv').drop('Unnamed: 0', axis=1)
dst = pd.read_csv('dst.csv').drop('Unnamed: 0', axis=1)


In [8]:
df = combined_csv.merge(orig, on='OriginState', how='left').merge(dst, on='DestState', how='left')

In [9]:
cols = ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateName',
 'DestAirportID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateName',
 'DepTime',
 'DepDelay',
 'ArrTime',
 'ArrDelay',
 'Cancelled',
 'Diverted',
 'ActualElapsedTime',
 'Flights',
 'Distance']

df = df[cols]

In [10]:
del combined_csv

In [12]:
import copy
DF_WITH_NA = copy.copy(df)
df.dropna(inplace=True)

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.to_csv('drive/MyDrive/BigData-finalproject/complete_data/2020_2019.csv', index=False)

# merge all data

In [ ]:
cd /content/drive/MyDrive/BigData-finalproject/complete_data/

/content/drive/MyDrive/BigData-finalproject/complete_data


In [ ]:
ls

2020_2019.csv  airline.tar.gz  clean_data.csv  subset/


In [ ]:
!head -n 2 2020_2019.csv

Year,Month,DayofMonth,DayOfWeek,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,DepTime,DepDelay,ArrTime,ArrDelay,Cancelled,Diverted,ActualElapsedTime,Flights,Distance
2019,2,1,5,AA,N110AN,1,12478,JFK,"New York, NY",NY,New York,12892,LAX,"Los Angeles, CA",CA,California,759.0,-1.0,1058.0,-35.0,0.0,0.0,359.0,1.0,2475.0


In [ ]:
!cat clean_data.csv <(tail +2 2020_2019.csv) > complete_clean_data.csv

# removing nulls resulted in all the cancelled flights being removed, I will need to take the cancellation data and save it separately in a different csv

In [14]:
DF_WITH_NA [['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'OriginAirportID',
 'OriginCityName',
 'OriginStateName',
 'DestAirportID',
 'DestCityName',
 'DestStateName',
 'Cancelled']].to_csv('withna2020.csv', index=False)

In [19]:
del DF_WITH_NA

In [17]:
!head -n 2 withna2020.csv

Year,Month,DayofMonth,DayOfWeek,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,Cancelled
2019,8,2,5,14057,"Portland, OR",Oregon,13487,"Minneapolis, MN",Minnesota,0.0


In [2]:
path = 'drive/MyDrive/BigData-finalproject/complete_data/subset/merged.csv'
df_rest= spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(path).persist()


In [3]:
df_rest = df_rest [['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'OriginAirportID',
 'OriginCityName',
 'OriginStateName',
 'DestAirportID',
 'DestCityName',
 'DestStateName',
 'Cancelled']].filter('Year > 1995') # .toPandas().to_csv('withnaall.csv', index=False)

In [6]:
df_rest.write.csv('withnaall.csv')

In [11]:
!cat withnaall.csv/*.csv > withna_all.csv


In [13]:
#add the new header
!sed  -i '1i Year,Month,DayofMonth,DayOfWeek,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,Cancelled' withna_all.csv


In [14]:
!head -n 2 withna_all.csv

Year,Month,DayofMonth,DayOfWeek,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,Cancelled
1996,1,24,3,14869,"Salt Lake City, UT",Utah,11292,"Denver, CO",Colorado,0.0


In [15]:
!cat withna_all.csv <(tail +2 withna2020.csv) > cancellation.csv

In [16]:
!cp cancellation.csv drive/MyDrive/BigData-finalproject/complete_data/

In [19]:
from pyspark.sql import functions as F
df_rest[['Cancelled']].agg(F.sum('Cancelled')).show()

+--------------+
|sum(Cancelled)|
+--------------+
|     1512877.0|
+--------------+

